# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
cities_data_path = 'output_data/cities.csv'

# Read the mouse data and the study results
cities_data = pd.read_csv(cities_data_path)

cities_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hobart,-42.8794,147.3294,285.93,87,75,8.23,AU,Sun May 2 22:28:33 2021
1,1,Uruaçu,-14.5247,-49.1408,293.85,70,3,1.51,BR,Sun May 2 22:28:34 2021
2,2,Mount Gambier,-37.8333,140.7667,288.15,82,75,8.23,AU,Sun May 2 22:28:35 2021
3,3,Bongandanga,1.5000,21.0500,294.58,89,100,1.03,CD,Sun May 2 22:28:37 2021
4,4,Hithadhoo,-0.6000,73.0833,301.77,71,98,5.47,MV,Sun May 2 22:28:38 2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(g_key)

locations = cities_data[["Lat", "Lng"]].astype(float)

humidity_rate = cities_data["Humidity"].astype(float)

In [19]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:


hotel_df = cities_data.loc[(cities_data["Max Temp"] > 158) & 
                           (cities_data["Wind Speed"] < 10) &
                                    (cities_data["Cloudiness"] == 0)]
hotel_df 


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,Port Macquarie,-31.4333,152.9167,297.59,69,0,0.89,AU,Sun May 2 22:28:44 2021
15,15,Port Alfred,-33.5906,26.8910,290.53,74,0,5.48,ZA,Sun May 2 22:28:46 2021
32,32,Port Elizabeth,-33.9180,25.5701,286.15,88,0,2.06,ZA,Sun May 2 22:28:51 2021
33,33,Saint-Pierre,-21.3393,55.4781,296.15,73,0,3.60,RE,Sun May 2 22:28:52 2021
38,38,Flinders,-34.5833,150.8552,298.71,63,0,0.89,AU,Sun May 2 22:28:54 2021
...,...,...,...,...,...,...,...,...,...,...
503,503,Faya,18.3851,42.4509,288.15,82,0,2.06,SA,Sun May 2 22:32:10 2021
516,516,Fatezh,52.0891,35.8632,287.15,88,0,7.00,RU,Sun May 2 22:32:16 2021
529,529,Palana,59.1167,159.9667,272.62,66,0,5.70,RU,Sun May 2 22:32:20 2021
538,538,Kiryat Gat,31.6100,34.7642,292.15,48,0,2.06,IL,Sun May 2 22:29:01 2021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df['Hotel Name'] = hotel_df
hotel_df['Hotel Address'] = hotel_df 

hotel_df


<ipython-input-25-7cecc50db67e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
10,10,Port Macquarie,-31.4333,152.9167,297.59,69,0,0.89,AU,Sun May 2 22:28:44 2021,10,Port Macquarie
15,15,Port Alfred,-33.5906,26.8910,290.53,74,0,5.48,ZA,Sun May 2 22:28:46 2021,15,Grahamstown
32,32,Port Elizabeth,-33.9180,25.5701,286.15,88,0,2.06,ZA,Sun May 2 22:28:51 2021,32,Gqeberha
33,33,Saint-Pierre,-21.3393,55.4781,296.15,73,0,3.60,RE,Sun May 2 22:28:52 2021,33,Saint-Denis
38,38,Flinders,-34.5833,150.8552,298.71,63,0,0.89,AU,Sun May 2 22:28:54 2021,38,Wollongong
...,...,...,...,...,...,...,...,...,...,...,...,...
503,503,Faya,18.3851,42.4509,288.15,82,0,2.06,SA,Sun May 2 22:32:10 2021,503,Abha
516,516,Fatezh,52.0891,35.8632,287.15,88,0,7.00,RU,Sun May 2 22:32:16 2021,516,Kursk
529,529,Palana,59.1167,159.9667,272.62,66,0,5.70,RU,Sun May 2 22:32:20 2021,529,Palana
538,538,Kiryat Gat,31.6100,34.7642,292.15,48,0,2.06,IL,Sun May 2 22:29:01 2021,538,Jerusalem


In [24]:
params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))